# Getty TGN | check on longitudes and latitudes by cross-checking other sources
- 2024-05-18
- Geo check on Getty TGN data fetched via SPARQL-endpoint
- https://amandinancy16.medium.com/reverse-geocoding-with-geopy-c26cfb63f74c
- V. Martens

# Import

## Import packages

In [8]:
# creating time stamps
import time

# importing files
import glob
import os

# progress bar
# from tqdm.notebook import tqdm
from tqdm import tqdm

# regex module
import re

# for multi-threading
from concurrent import futures
from concurrent.futures import ThreadPoolExecutor
import multiprocessing as mp

# retrieve country names from country codes
import pycountry

# data wrangling
import pandas as pd
import numpy as np

# back up files
import pickle

# retrieve various geo locations
import reverse_geocoder as rg
from geopy.geocoders import Nominatim

# measure distance between two geo points
from haversine import haversine, Unit

# save and dump
import pickle

# surpress, warnings, uhoohhhh
import warnings
warnings.filterwarnings("ignore")

# preferences
# adjust pandas to show all cols
pd.set_option('display.max_colwidth', None)

## Import functions

In [19]:
def load_latest_file(filepath:str) -> str:
    '''
    loads created latest create file from a directory
    args: string with filepath
    returns: latest file from a list of files
    '''

    list_of_files = glob.glob(filepath)
    latest_file = max(list_of_files, key=os.path.getctime)
    print(latest_file)
    
    return latest_file

def join_lats_lons(df:pd.DataFrame) -> pd.DataFrame:
    '''
    args: pd.DataFrame with lats and lons
    returns: pd.DataFrame with corrected lats and lons and joined into one col 
    '''
    
    df['lat'] = df['lat'].str.replace('-.','-0.')
    df['lon'] = df['lon'].str.replace('-.','-0.')
    df['coordinates'] = list(zip(df['lat'], df['lon']))
    
    return df

def get_country(df:pd.DataFrame, i:int) -> list:
    '''
    args: pd.DataFrame() with coordinates column
    returns: a list with dicts with geo locations, every dict contains keys with country name, 3 admin names, lat, lon
    '''
    
    results = rg.search(df['coordinates'].iloc[i])

    for item in results:
        query = df['coordinates'].iloc[i]
        tgn_id = df['tgn_id'].iloc[i]
        country = item['cc']
        looked_lat = item['lat']
        looked_lon = item['lon']
        looked_name = item['name']
        
    return [tgn_id, query, country, looked_lat, looked_lon, looked_name]

def parse_reverse_geo_lookup(list_of_dicts:list) -> pd.DataFrame:
    '''
    args: a list with dicts with geo locations
    returns: pd.DataFrame() with parsed geo locations
    '''
    
    dfs = pd.DataFrame()

    for coord_dict in list_of_dicts:
        df = pd.DataFrame(coord_dict).T
        dfs = pd.concat([dfs, df])
    
    dfs = dfs.rename(columns={0 : 'tgn_id_lookup',
                              1 : 'query_look_up',
                              2 : 'country_code_lookup', 
                              3 : 'lat_lookup',
                              4 : 'lon_lookup', 
                              5 : 'municipality_lookup'})
    
    return dfs
    
def parse_parentstring(df:pd.DataFrame) -> pd.DataFrame:
    '''
    parses out info from parentstring
    args: pd.DataFrame() with tgn parentstrings
    returns: pd.DataFrame() with reordered and parsed columns
    '''
    
    df['parentstring_province'] = (df['parentstring']
                                    .str.split(',')
                                    .str[0]
                                    .str.strip())
    
    df['parentstring_country'] = (df['parentstring']
                                    .str.split(',')
                                    .str[1]
                                    .str.strip())
    
    df['parentstring_continent'] = (df['parentstring']
                                    .str.split(',')
                                    .str[2]
                                    .str.strip())  
    
    df['parentstring_world'] = (df['parentstring']
                                    .str.split(',')
                                    .str[3]
                                    .str.strip())  
    
    df = df[['tgn_id', 'city_name', 'inferred_city_name', 'parentstring_province',
             'parentstring_country', 'parentstring_continent', 'parentstring_world',
             'broader_parentstring', 'lat', 'lon', 'coordinates', 'query_look_up',
             'country_code_lookup', 'lat_lookup', 'lon_lookup',
             'municipality_lookup']]
    
    return df    

def parse_broader_parentstring(df:pd.DataFrame) -> pd.DataFrame:
    '''
    parses out info from broader parentstring
    args: pd.DataFrame() with tgn broader parentstrings
    returns: pd.DataFrame() with reordered and parsed columns
    '''
    
    df['broader_parentstring_country'] = (df['broader_parentstring']
                                    .str.split()
                                    .str[0]
                                    .str.replace(',','')
                                    .str.strip())
    
    df['broader_parentstring_continent'] = (df['broader_parentstring']
                                    .str.split()
                                    .str[1]
                                    .str.replace(',','')
                                    .str.strip())  
    
    df['broader_parentstring_world'] = (df['broader_parentstring']
                                    .str.split()
                                    .str[2]
                                    .str.replace(',','')
                                    .str.strip())  
    
    df = df[['tgn_id', 'city_name', 'inferred_city_name', 'parentstring_province',
       'parentstring_country', 'parentstring_continent', 'parentstring_world',
       'broader_parentstring_country', 'broader_parentstring_continent',
       'broader_parentstring_world', 'lat', 'lon', 'coordinates', 'query_look_up',
       'country_code_lookup', 'lat_lookup', 'lon_lookup', 'municipality_lookup']]
    
    return df

def country_code_lookup(country_code:str) -> str:
    '''
    
    retrieves countryname from countrycode
    args: string with countrycode
    returns: returns country name
    '''
    
    return pycountry.countries.get(alpha_2=country_code).name

def municipality_look_up(df:pd.DataFrame) -> dict:
    '''
    pd.DataFrame with city names retrieves city info, lats, lons
    args: pd.DataFrame with city name/municipalities
    returns: a dict with city names as keys and location info and lats, lons as values
    '''
    
    geolocator = Nominatim(user_agent="Nancy Amandi", timeout=10)
    
    dict_locs = {}
    
    for i in tqdm(range(len(df)), total=len(df), desc='find geo info, based on city names'):
        
        # lambda query: geolocator.geocode("%s, Cleveland OH" % query)
        
        dict_locs[df['municipality_lookup'].iloc[i]] = lambda query: geolocator.geocode(f"{df['municipality_lookup'].iloc[i]}, NL")
    
    return dict_locs

def parse_geo_lookup(results_lookup:dict) -> pd.DataFrame:
    '''
    parses looked up geo data into a pd.DataFrame
    args: a dict with looked up geo locations based on city names
    returns: a pd.DataFrame with city names, lat-lons
    '''
    
    df_parsed_geo = (pd.DataFrame(results_lookup)
                    .T.reset_index()
                    .rename(columns={'index':'query_city_name',
                                    0 : 'query_city_city_info',
                                    1 : 'query_city_geo_data'}))
    
    return df_parsed_geo

def data_haversine_tuple_creator(df:pd.DataFrame) -> pd.DataFrame:
    '''
    args: pd.DataFrame with seperate lat lon
    returns: pd.Dataframe with a tuple of lat lon in one col
    '''
    
    df['coordinates_looked_up'] = list(zip(df['lat_lookup'].astype(float), df['lon_lookup'].astype(float)))
    df = df.drop(columns=['lat_lookup', 'lon_lookup'])
    
    return df

def haversine_distance_calculator(df) -> pd.DataFrame:
    '''
    measures distance between two tuples containing two geo points each 
    args: a pd.DataFrame with two columns containing tuples with floats
    returns: a pd.DataFrame with a column that has measured the difference in kms between two tuples with geo points
    '''
    
    df['km_difference'] = '' 
    
    for i in tqdm(range(len(df)), total=len(df), desc='calculate diff between geo points'):
        df['km_difference'].iloc[i] = round(haversine(df['query_city_geo_data'].iloc[i], df['coordinates_looked_up'].iloc[i], unit=Unit.KILOMETERS),2)
        
    return df

def lon_lat_diff(df:pd.DataFrame) -> pd.DataFrame:
    '''
    diff betweeon geo points
    args: pd.DataFrame with two lats and two lons as a lats + lons tuple
    returns: pd.DataFrame pd.DataFrame with diff between lats and lons combination
    '''
    
    df['lat_diff'] = ''
    df['lon_diff'] = ''

    for i in tqdm(range(len(df)), total=len(df), desc='calculate lats and lons differences'):
        df['lat_diff'].iloc[i] = float(df['query_look_up'][i][0]) - float(df['query_city_geo_data'][i][0])
        df['lon_diff'].iloc[i] = float(df['query_look_up'][i][1]) - float(df['query_city_geo_data'][i][1])
        
    return df

## Constants + user input

In [ ]:
# country_check 
country = 'Netherlands'
continent = 'Europe'

# create back up filename for a pickle
time_stamp = time.strftime('%Y%m%d-%H%M%S')
filename_df_errors = f'{time_stamp}_df_errors_tgn.pickle'

# create back up filename for a pickle
time_stamp = time.strftime('%Y%m%d-%H%M%S')
filename_df_lod_results = f'{time_stamp}_df_lod_results_tgn.pickle'

# create back up filename for a pickle
time_stamp = time.strftime('%Y%m%d-%H%M%S')
filename_excel_export = f'{time_stamp}_{country}_loc_check_TGN.xlsx'

print(f"{filename_df_errors}, {filename_df_lod_results}, {filename_excel_export}")

20240607-143032_df_errors_tgn.pickle, 20240607-143032_df_lod_results_tgn.pickle, 20240607-143032_loc_check_TGN.xlsx


## Import data

In [ ]:
latest_picke_file = load_latest_file('data_dumps/*tgn_country.pickle')

# Open the file in binary mode
with open(latest_picke_file, 'rb') as file:
      
    # Call load method to deserialze
    df_tgn = pickle.load(file)
    
# enrich lats, lons
df_tgn = join_lats_lons(df_tgn)
df_tgn.shape



data_dumps\results_df_tgn_country.pickle


# Subset data

In [ ]:
# subset data
df_nl = df_tgn[(df_tgn['broader_parentstring'].str.contains(f'{country}') == True)]
df_europe = df_tgn[(df_tgn['broader_parentstring'].str.contains(f'{europe}') == True)]
df_nl.shape, df_europe.shape

# Geo data-check
 1. Reverse city look-up on geo lats + lons 
 1. Retrieve lats, lons based on reversed look-up
 1. Compare distances between two geo-points

In [7]:
%%time

import multiprocessing as mp
pools = mp.cpu_count()

# Start pool
thread_pool = ThreadPoolExecutor(max_workers=pools, thread_name_prefix = 'thread')

# reate futures
futures = [thread_pool.submit(get_country, df_nl, i) for i in range(len())]

# submit tasks
results = [future.result() for future in tqdm(futures, total=len(futures), desc='find reverse geo codes')]

# reverse geo lookup to a df
df_geo_data = parse_reverse_geo_lookup(results)

# merge results geo lookop with main tgn dataset
 = (pd.merge(df_nl, df_geo_data, left_on='tgn_id', right_on='tgn_id_lookup', how='left')
         .drop(columns=['tgn_id_lookup'])
         )

# parse tgn dataset into seperate cols
df_nl = parse_parentstring(df_nl)
df_nl = parse_broader_parentstring(df_nl)

# change country code into country name to allign with tgn data
df_nl['country_name_lookup'] = (df_nl['country_code_lookup']
                                .apply(country_code_lookup)
                                )

del df_nl['country_code_lookup']

# look up lat lons based on reverse geo lookup and return lat lons
results_lookup = municipality_look_up(df_nl)

# parse geo lookup into df
df_results = parse_geo_lookup(results_lookup)

# merge geo lookup with main tgn dataset
df_nl = (pd.merge(df_nl, df_results, left_on='municipality_lookup', right_on='query_city_name', how='left')
         .drop(columns=['query_city_name'])
         )

# creates a col with coordinate tuples for haversine distance measurements
df_nl = data_haversine_tuple_creator(df_nl)

# measures differences between two geo locations
df_nl = haversine_distance_calculator(df_nl)

# diff between lots and lats combinations
df_nl = lon_lat_diff(df_nl)

df_nl.head()

Loading formatted geocoded file...
Loading formatted geocoded file...
Loading formatted geocoded file...
Loading formatted geocoded file...
Loading formatted geocoded file...
Loading formatted geocoded file...
Loading formatted geocoded file...
Loading formatted geocoded file...


find geo info based on city names: 100%|██████████| 1951/1951 [00:00<00:00, 3097.58it/s]

CPU times: total: 1min 31s
Wall time: 1h 25min 11s


,tgn_id,city_name,inferred_city_name,parentstring_province,parentstring_country,parentstring_continent,parentstring_world,broader_parentstring_country,broader_parentstring_continent,broader_parentstring_world,lat,lon,coordinates,query_look_up,municipality_lookup,country_name_lookup,query_city_city_info,query_city_geo_data,coordinates_looked_up,km_difference
0,http://vocab.getty.edu/tgn/7257647,Aagtekerke,Zeeland,Zeeland,Netherlands,Europe,World,Netherlands,Europe,World,51.55,3.516667,"(51.55, 3.516667)","(51.55, 3.516667)",Middelburg,Netherlands,"Middelburg, Zeeland, Nederland","(51.4996784, 3.6137371)","(51.5, 3.61389)",0.04
1,http://vocab.getty.edu/tgn/7257816,Aardam,South Holland,South Holland,Netherlands,Europe,World,Netherlands,Europe,World,52.166667,4.766667,"(52.166667, 4.766667)","(52.166667, 4.766667)",Nieuwkoop,Netherlands,"Nieuwkoop, Zuid-Holland, Nederland","(52.1500947, 4.7761706)","(52.15083, 4.77639)",0.08
2,http://vocab.getty.edu/tgn/7257652,Aa,Utrecht,Utrecht,Netherlands,Europe,World,Netherlands,Europe,World,52.166667,4.983333,"(52.166667, 4.983333)","(52.166667, 4.983333)",Breukelen,Netherlands,"Breukelen, Stichtse Vecht, Utrecht, Nederland","(52.171959, 5.002675)","(52.17417, 5.00139)",0.26
3,http://vocab.getty.edu/tgn/7257651,Aadorp,Overijssel,Overijssel,Netherlands,Europe,World,Netherlands,Europe,World,52.366667,6.633333,"(52.366667, 6.633333)","(52.366667, 6.633333)",Almelo,Netherlands,"Almelo, Overijssel, Nederland","(52.3567956, 6.66725277634109)","(52.35667, 6.6625)",0.32
4,http://vocab.getty.edu/tgn/7257653,Aa,North Brabant,North Brabant,Netherlands,Europe,World,Netherlands,Europe,World,51.65,5.316667,"(51.65, 5.316667)","(51.65, 5.316667)",Vught,Netherlands,"Vught, Noord-Brabant, Nederland","(51.6511806, 5.23680014410728)","(51.65333, 5.2875)",3.51


In [15]:
df_nl = lon_lat_diff(df_nl)
df_nl.head()

calculate lats and lons differences: 100%|██████████| 1951/1951 [00:01<00:00, 1720.67it/s]


,tgn_id,city_name,inferred_city_name,parentstring_province,parentstring_country,parentstring_continent,parentstring_world,broader_parentstring_country,broader_parentstring_continent,broader_parentstring_world,...,coordinates,query_look_up,municipality_lookup,country_name_lookup,query_city_city_info,query_city_geo_data,coordinates_looked_up,km_difference,lat_diff,lon_diff
0,http://vocab.getty.edu/tgn/7257647,Aagtekerke,Zeeland,Zeeland,Netherlands,Europe,World,Netherlands,Europe,World,...,"(51.55, 3.516667)","(51.55, 3.516667)",Middelburg,Netherlands,"Middelburg, Zeeland, Nederland","(51.4996784, 3.6137371)","(51.5, 3.61389)",0.04,0.050322,-0.09707
1,http://vocab.getty.edu/tgn/7257816,Aardam,South Holland,South Holland,Netherlands,Europe,World,Netherlands,Europe,World,...,"(52.166667, 4.766667)","(52.166667, 4.766667)",Nieuwkoop,Netherlands,"Nieuwkoop, Zuid-Holland, Nederland","(52.1500947, 4.7761706)","(52.15083, 4.77639)",0.08,0.016572,-0.009504
2,http://vocab.getty.edu/tgn/7257652,Aa,Utrecht,Utrecht,Netherlands,Europe,World,Netherlands,Europe,World,...,"(52.166667, 4.983333)","(52.166667, 4.983333)",Breukelen,Netherlands,"Breukelen, Stichtse Vecht, Utrecht, Nederland","(52.171959, 5.002675)","(52.17417, 5.00139)",0.26,-0.005292,-0.019342
3,http://vocab.getty.edu/tgn/7257651,Aadorp,Overijssel,Overijssel,Netherlands,Europe,World,Netherlands,Europe,World,...,"(52.366667, 6.633333)","(52.366667, 6.633333)",Almelo,Netherlands,"Almelo, Overijssel, Nederland","(52.3567956, 6.66725277634109)","(52.35667, 6.6625)",0.32,0.009871,-0.03392
4,http://vocab.getty.edu/tgn/7257653,Aa,North Brabant,North Brabant,Netherlands,Europe,World,Netherlands,Europe,World,...,"(51.65, 5.316667)","(51.65, 5.316667)",Vught,Netherlands,"Vught, Noord-Brabant, Nederland","(51.6511806, 5.23680014410728)","(51.65333, 5.2875)",3.51,-0.001181,0.079867


# Back-up data

In [16]:
with open(f'data_dumps/{filename_excel_export}.pickle', 'wb') as handle:
    pickle.dump(df_nl, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Geo location checks

In [18]:
df_nl[(df_nl['km_difference'] > 5)]

,tgn_id,city_name,inferred_city_name,parentstring_province,parentstring_country,parentstring_continent,parentstring_world,broader_parentstring_country,broader_parentstring_continent,broader_parentstring_world,...,coordinates,query_look_up,municipality_lookup,country_name_lookup,query_city_city_info,query_city_geo_data,coordinates_looked_up,km_difference,lat_diff,lon_diff
5,http://vocab.getty.edu/tgn/7270017,Aabeek,Limburg,Limburg,Netherlands,Europe,World,Netherlands,Europe,World,...,"(51.25, 6)","(51.25, 6)",Leeuwen,Netherlands,"Leeuwen, Wageningen-Hoog, Wageningen, Gelderland, Nederland, 6704 AN, Nederland","(51.9826484, 5.6758503)","(51.21032, 5.99862)",88.73,-0.732648,0.32415
8,http://vocab.getty.edu/tgn/7258113,Abtstocht,Flevoland,Flevoland,Netherlands,Europe,World,Netherlands,Europe,World,...,"(52.616667, 5.683333)","(52.616667, 5.683333)",Urk,Netherlands,"Urk, Flevoland, Nederland","(52.6927959, 5.457617050455788)","(52.6625, 5.60139)",10.26,-0.076129,0.225716
13,http://vocab.getty.edu/tgn/7257649,Aagtdorp,North Holland,North Holland,Netherlands,Europe,World,Netherlands,Europe,World,...,"(52.683333, 4.7)","(52.683333, 4.7)",Alkmaar,Netherlands,"Alkmaar, Noord-Holland, Nederland","(52.600853799999996, 4.817099446443963)","(52.63167, 4.74861)",5.76,0.082479,-0.117099
17,http://vocab.getty.edu/tgn/4006946,Marken,Marken island,Marken island,Flevoland,Netherlands,Europe,Flevoland,Netherlands,Europe,...,"(52.466667, 5.1)","(52.466667, 5.1)",Marken,Netherlands,"Marche, Italia","(43.3458388, 13.1415872)","(52.45833, 5.10278)",1175.56,9.120828,-8.041587
26,http://vocab.getty.edu/tgn/7006756,Arnemuiden,Zuid Beveland,Zuid Beveland,Zeeland,Netherlands,Europe,Zeeland,Netherlands,Europe,...,"(51.5, 3.666667)","(51.5, 3.666667)",Veere,Netherlands,"Veere, Zeeland, Nederland","(51.55640385, 3.577269355747127)","(51.54833, 3.66667)",6.25,-0.056404,0.089398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1929,http://vocab.getty.edu/tgn/7271340,Schokkerbos,Flevoland,Flevoland,Netherlands,Europe,World,Netherlands,Europe,World,...,"(52.65, 5.766667)","(52.65, 5.766667)",Ens,Netherlands,"Ens, Bagnères-de-Bigorre, Hautes-Pyrénées, Occitanie, France métropolitaine, 65170, France","(42.8063652, 0.3402903)","(52.63667, 5.82778)",1166.77,9.843635,5.426377
1934,http://vocab.getty.edu/tgn/7256164,Berkt,North Brabant,North Brabant,Netherlands,Europe,World,Netherlands,Europe,World,...,"(51.433333, 5.383333)","(51.433333, 5.383333)",Oerle,Netherlands,"Oreye, Waremme, Liège, Wallonie, 4360, België / Belgique / Belgien","(50.7291681, 5.3530675)","(51.42255, 5.37163)",77.11,0.704165,0.030266
1938,http://vocab.getty.edu/tgn/7261092,De Noord,Zeeland,Zeeland,Netherlands,Europe,World,Netherlands,Europe,World,...,"(51.516667, 4.166667)","(51.516667, 4.166667)",Tholen,Netherlands,"Tholen, Zeeland, Nederland","(51.57085015, 4.076142821673509)","(51.53167, 4.22083)",10.91,-0.054183,0.090524
1941,http://vocab.getty.edu/tgn/7271339,Schokkerhaven,Flevoland,Flevoland,Netherlands,Europe,World,Netherlands,Europe,World,...,"(52.616667, 5.75)","(52.616667, 5.75)",Ens,Netherlands,"Ens, Bagnères-de-Bigorre, Hautes-Pyrénées, Occitanie, France métropolitaine, 65170, France","(42.8063652, 0.3402903)","(52.63667, 5.82778)",1166.77,9.810302,5.40971


In [24]:
df_nl[(df_nl['km_difference'] > 5) & ((df_nl['lat_diff'] >= 0.5) & (df_nl['lat_diff'] < 1.5))]

,tgn_id,city_name,inferred_city_name,parentstring_province,parentstring_country,parentstring_continent,parentstring_world,broader_parentstring_country,broader_parentstring_continent,broader_parentstring_world,...,coordinates,query_look_up,municipality_lookup,country_name_lookup,query_city_city_info,query_city_geo_data,coordinates_looked_up,km_difference,lat_diff,lon_diff
479,http://vocab.getty.edu/tgn/7258274,Achterste Stroom,North Brabant,North Brabant,Netherlands,Europe,World,Netherlands,Europe,World,...,"(51.583333, 5.216667)","(51.583333, 5.216667)",Haaren,Netherlands,"Haren, Ville de Bruxelles - Stad Brussel, Brussel-Hoofdstad - Bruxelles-Capitale, Région de Bruxelles-Capitale - Brussels Hoofdstedelijk Gewest, 1110, België / Belgique / Belgien","(50.8909441, 4.4157553)","(51.6025, 5.22222)",97.01,0.692389,0.800912
649,http://vocab.getty.edu/tgn/7258431,Beijum,Groningen,Groningen,Netherlands,Europe,World,Netherlands,Europe,World,...,"(53.25, 6.6)","(53.25, 6.6)",Oosterpark,Netherlands,"Oosterpark, Oost, Amsterdam, Noord-Holland, Nederland","(52.36014335, 4.920540276968429)","(53.2237, 6.5853)",147.48,0.889857,1.67946
791,http://vocab.getty.edu/tgn/7256356,Anny's Hoeve,North Brabant,North Brabant,Netherlands,Europe,World,Netherlands,Europe,World,...,"(51.666667, 5.166667)","(51.666667, 5.166667)",Heusden,Netherlands,"Hesdin, Montreuil-sur-Mer, Pas-de-Calais, Hauts-de-France, France métropolitaine, 62140, France","(50.3730005, 2.0366141)","(51.73417, 5.13889)",264.4,1.293666,3.130053
828,http://vocab.getty.edu/tgn/7258609,Bemelen,Limburg,Limburg,Netherlands,Europe,World,Netherlands,Europe,World,...,"(50.85, 5.766667)","(50.85, 5.766667)",Berg,Netherlands,"Berg-sur-Moselle, Thionville, Moselle, Grand Est, France métropolitaine, 57570, France","(49.4307696, 6.3111933)","(50.86167, 5.78333)",163.49,1.41923,-0.544526
840,http://vocab.getty.edu/tgn/7258607,Bemelerberg,Limburg,Limburg,Netherlands,Europe,World,Netherlands,Europe,World,...,"(50.85, 5.766667)","(50.85, 5.766667)",Berg,Netherlands,"Berg-sur-Moselle, Thionville, Moselle, Grand Est, France métropolitaine, 57570, France","(49.4307696, 6.3111933)","(50.86167, 5.78333)",163.49,1.41923,-0.544526
897,http://vocab.getty.edu/tgn/7262348,Brandenburg,Utrecht,Utrecht,Netherlands,Europe,World,Netherlands,Europe,World,...,"(52.183333, 5.25)","(52.183333, 5.25)",Soest,Netherlands,"Soest, Kreis Soest, Nordrhein-Westfalen, 59494, Deutschland","(51.5725501, 8.1061259)","(52.17333, 5.29167)",204.43,0.610783,-2.856126
1089,http://vocab.getty.edu/tgn/7262680,Bunt,Utrecht,Utrecht,Netherlands,Europe,World,Netherlands,Europe,World,...,"(52.15, 5.333333)","(52.15, 5.333333)",Soest,Netherlands,"Soest, Kreis Soest, Nordrhein-Westfalen, 59494, Deutschland","(51.5725501, 8.1061259)","(52.17333, 5.29167)",204.43,0.57745,-2.772793
1229,http://vocab.getty.edu/tgn/7006977,Bourtange,Groningen,Groningen,Netherlands,Europe,World,Netherlands,Europe,World,...,"(53.016667, 7.2)","(53.016667, 7.2)",Rhede,Germany,"Rhede, Kreis Borken, Nordrhein-Westfalen, 46414, Deutschland","(51.8363135, 6.696091)","(53.0602, 7.27127)",141.56,1.180353,0.503909
1278,http://vocab.getty.edu/tgn/7262998,De Birk,Utrecht,Utrecht,Netherlands,Europe,World,Netherlands,Europe,World,...,"(52.166667, 5.333333)","(52.166667, 5.333333)",Soest,Netherlands,"Soest, Kreis Soest, Nordrhein-Westfalen, 59494, Deutschland","(51.5725501, 8.1061259)","(52.17333, 5.29167)",204.43,0.594117,-2.772793
1564,http://vocab.getty.edu/tgn/7261290,De Paltz,Utrecht,Utrecht,Netherlands,Europe,World,Netherlands,Europe,World,...,"(52.133333, 5.283333)","(52.133333, 5.283333)",Soest,Netherlands,"Soest, Kreis Soest, Nordrhein-Westfalen, 59494, Deutschland","(51.5725501, 8.1061259)","(52.17333, 5.29167)",204.43,0.560783,-2.822793


# Other options
- with geopy package

In [47]:
import concurrent.futures
import multiprocessing as mp
from functools import partial
from pprint import pprint

cpu_count_laptop = mp.cpu_count()
geolocator = Nominatim(user_agent="Nancy Amandi", timeout=10)
geocode = RateLimiter(geolocator, swallow_exceptions=True, min_delay_seconds=0.1, return_value_on_exception=None) 

with concurrent.futures.ThreadPoolExecutor(max_workers=cpu_count_laptop, thread_name_prefix='thread') as e:
    locations = list(e.map(partial(geocode, language='en', exactly_one=True), tqdm(list(set(df_nl["municipality_lookup"]))), chunksize=100))
    
for i in range(len(locations)):
    # pprint(locations[i].raw)
    print(locations[i].raw['address']['country'])
    # print(locations[i].raw['address']['city'])

100%|██████████| 5/5 [00:00<00:00, 1670.64it/s]


TypeError: 'Nominatim' object is not callable

In [46]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
tqdm.pandas()

# option 1
geolocator = Nominatim(user_agent="geopy.geocoders.Nominatim")
geocode = RateLimiter(geolocator, min_delay_seconds=1)
# df_nl['new_location'] = df_nl['municipality_lookup'].progress_apply(geocode)

# option 2
geolocator = Nominatim(user_agent="Nancy Amandi", timeout=10)
geocode = RateLimiter(geolocator.reverse, min_delay_seconds=0.1)
df_nl['new_location'] = df_nl['municipality_lookup'].apply(geocode, language='en', exactly_one=True )

# optione 3
df_tgn['address'] = df_tgn.progress_apply(lambda row: geocode((row['lat.value'], row['long.value']), language='en', exactly_one=True), axis=1)
df_tgn['country'] = df_tgn['address'].astype(str).str.split(',').str[-1]

# option 4
geolocator = Nominatim(user_agent="Nancy Amandi", timeout= 10)
rgeocode = RateLimiter(geolocator.reverse, min_delay_seconds=0.1)
df_tgn["location"] = df_tgn["coordinates"].progress_apply(rgeocode)

# func to work with try and except on multi-threaded geocode, doube apply
def eval_results(x):
    try:
        return (x.latitude, x.longitude)
    except:
        return (None, None)


TypeError: 'Nominatim' object is not callable